# Property Purchase Fees Calculator

Calculate UK property purchase costs including SDLT (Stamp Duty Land Tax) and deposits.

## Features
- Calculate SDLT with first-time buyer relief
- Compare different deposit percentages
- Analyze property affordability across price ranges
- Find maximum affordable property price based on savings and loan constraints

In [1]:
from tabulate import tabulate
from assetreturns import calculateSDLT

## Core Functions

In [2]:
def generate_property_table(leverage):
    """
    Generate table showing property prices and required funds.
    
    Args:
        leverage: Down payment percentage (e.g., 0.26 for 26% down payment)
    
    Returns:
        List of lists containing property price, SDLT, deposit, 
        total initial funds, and funds/cost ratio
    """
    table = []
    for property_price in range(800000, 1500000 + 100000, 10000):
        sdlt = calculateSDLT(second_property=False, property_value=property_price, is_first_time_buyer=True)
        deposit = property_price * leverage
        total_initial_funds = sdlt + deposit
        table.append([
            property_price,
            sdlt,
            deposit,
            total_initial_funds,
            total_initial_funds / property_price
        ])
    return table


def print_required_funds(leverage):
    """
    Print formatted table of required funds for various property prices.
    
    Args:
        leverage: Down payment percentage (e.g., 0.50 for 50% down payment)
    """
    table = generate_property_table(leverage)
    headers = [
        "Property Price",
        "SDLT",
        f"Deposit ({leverage*100:.0f}%)",
        "SDLT + Deposit",
        "Funds/Cost Ratio"
    ]
    print(tabulate(table, headers, tablefmt="presto"))


def compare_deposit_percentages(property_price, deposit_percentages, interest_rates=None, is_first_time_buyer=True):
    """
    Compare different deposit percentages for a specific property price.
    
    Args:
        property_price: Property price to analyze
        deposit_percentages: List of deposit percentages (e.g., [0.05, 0.10, 0.15, 0.20])
        interest_rates: List of annual interest rates (one per deposit %, e.g., [0.0509, 0.0483, 0.0450, 0.0391])
                       If None, monthly payment and marginal return columns are omitted
        is_first_time_buyer: Whether first-time buyer relief applies (default: True)
    
    Returns:
        Formatted table comparing different deposit scenarios including marginal returns
    """
    sdlt = calculateSDLT(second_property=False, property_value=property_price, is_first_time_buyer=is_first_time_buyer)
    
    # Validate interest_rates length if provided
    if interest_rates is not None and len(interest_rates) != len(deposit_percentages):
        raise ValueError(f"interest_rates length ({len(interest_rates)}) must match deposit_percentages length ({len(deposit_percentages)})")
    
    # Track previous row for marginal return calculation
    prev_monthly_payment = None
    prev_total_initial = None
    
    table = []
    for i, deposit_pct in enumerate(deposit_percentages):
        deposit = property_price * deposit_pct
        loan = property_price - deposit
        total_initial_funds = sdlt + deposit
        
        row = [
            f"{deposit_pct*100:.0f}%",
            f"£{deposit:,.0f}",
            f"£{loan:,.0f}",
        ]
        
        # Add interest rate and monthly payment if rates provided
        monthly_payment_io = None
        if interest_rates is not None:
            annual_rate = interest_rates[i]
            monthly_payment_io = (loan * annual_rate) / 12
            row.extend([
                f"{annual_rate*100:.2f}%",
                f"£{monthly_payment_io:,.0f}"
            ])
            
            # Calculate marginal return on extra deposit
            if i == 0:
                # First row - no previous row to compare
                marginal_return_str = "-"
            else:
                # Calculate return on extra deposit
                monthly_savings = prev_monthly_payment - monthly_payment_io
                extra_upfront = total_initial_funds - prev_total_initial
                
                if extra_upfront > 0:
                    marginal_annual_return = (monthly_savings * 12 / extra_upfront) * 100
                    marginal_return_str = f"{marginal_annual_return:.2f}%"
                else:
                    marginal_return_str = "N/A"
            
            row.append(marginal_return_str)
            
            # Store for next iteration
            prev_monthly_payment = monthly_payment_io
            prev_total_initial = total_initial_funds
        
        row.extend([
            f"£{total_initial_funds:,.0f}",
            f"{total_initial_funds / property_price:.2%}"
        ])
        
        table.append(row)
    
    # Build headers based on whether rates are included
    headers = [
        "Deposit %",
        "Deposit Amount",
        "Loan Amount",
    ]
    
    if interest_rates is not None:
        headers.extend([
            "Rate",
            "Monthly (IO)",
            "Return on Extra Deposit"
        ])
    
    headers.extend([
        "SDLT + Deposit",
        "Funds/Cost Ratio"
    ])
    
    print(f"\n{'='*80}")
    print(f"Property Price: £{property_price:,}")
    print(f"SDLT: £{sdlt:,.0f} {'(First-time buyer relief)' if is_first_time_buyer else ''}")
    print(f"{'='*80}")
    print(tabulate(table, headers, tablefmt="presto"))


def show_current_market_rates():
    """
    Display reference table of current UK mortgage rates (February 2026).
    
    Shows typical rates by LTV ratio for both 2-year and 5-year fixed mortgages.
    Use these as guidance when inputting interest rates into other functions.
    """
    print("\n" + "="*80)
    print("UK MORTGAGE RATES REFERENCE (February 2026)")
    print("="*80)
    print("\nBank of England Base Rate: 3.75%\n")
    
    rate_data = [
        ["60%", "3.91%", "4.00%"],
        ["75%", "~4.50%", "~4.60%"],
        ["80%", "4.83%", "4.91%"],
        ["90%", "5.09%", "5.07%"],
        ["95%", "4.84%", "4.77%"]
    ]
    
    headers = ["LTV Ratio", "2-Year Fixed", "5-Year Fixed"]
    print(tabulate(rate_data, headers, tablefmt="presto"))
    
    print("\n" + "-"*80)
    print("Note: Actual rates vary by lender, product type, credit score, and market")
    print("conditions. Check comparison sites for current offers:")
    print("  • MoneySavingExpert (moneysavingexpert.com)")
    print("  • MoneySuperMarket (moneysupermarket.com)")
    print("  • Compare the Market (comparethemarket.com)")
    print("="*80 + "\n")


def find_max_property_price(savings, leverage_percentage, max_nominal_leverage):
    """
    Find maximum affordable property price given constraints.
    
    Args:
        savings: Available savings amount
        leverage_percentage: Down payment percentage
        max_nominal_leverage: Maximum absolute loan amount
    
    Returns:
        Maximum affordable property price
    """
    table = generate_property_table(leverage_percentage)
    result = None
    
    for row in table:
        total_funds_required = row[3]
        nominal_leverage = row[0] - row[2]
        
        if nominal_leverage > max_nominal_leverage:
            break
        elif total_funds_required < savings:
            result = row
        else:
            break
    
    return result[0] if result else None

## Example 0: Property Price Range Analysis

Analyze required funds across a range of property prices with a fixed deposit percentage.

In [3]:
# Example: 10% deposit across property price range
print("Property Price Analysis with 10% Deposit:")
print_required_funds(0.1)

Property Price Analysis with 10% Deposit:
   Property Price |   SDLT |   Deposit (10%) |   SDLT + Deposit |   Funds/Cost Ratio
------------------+--------+-----------------+------------------+--------------------
           800000 |  30000 |           80000 |           110000 |           0.1375
           810000 |  30500 |           81000 |           111500 |           0.137654
           820000 |  31000 |           82000 |           113000 |           0.137805
           830000 |  31500 |           83000 |           114500 |           0.137952
           840000 |  32000 |           84000 |           116000 |           0.138095
           850000 |  32500 |           85000 |           117500 |           0.138235
           860000 |  33000 |           86000 |           119000 |           0.138372
           870000 |  33500 |           87000 |           120500 |           0.138506
           880000 |  34000 |           88000 |           122000 |           0.138636
           890000 |  345

# Example 1: Current Market Rates Reference

View current UK mortgage rates by LTV ratio for guidance when selecting interest rates.

In [4]:
# Display current UK mortgage rates for reference
show_current_market_rates()


UK MORTGAGE RATES REFERENCE (February 2026)

Bank of England Base Rate: 3.75%

 LTV Ratio   | 2-Year Fixed   | 5-Year Fixed
-------------+----------------+----------------
 60%         | 3.91%          | 4.00%
 75%         | ~4.50%         | ~4.60%
 80%         | 4.83%          | 4.91%
 90%         | 5.09%          | 5.07%
 95%         | 4.84%          | 4.77%

--------------------------------------------------------------------------------
Note: Actual rates vary by lender, product type, credit score, and market
conditions. Check comparison sites for current offers:
  • MoneySavingExpert (moneysavingexpert.com)
  • MoneySuperMarket (moneysupermarket.com)
  • Compare the Market (comparethemarket.com)



## Example 2: Deposit Percentage Comparison

Compare different deposit percentages for a specific property price.

In [5]:
# Example: Compare deposit options for £800,000 property
# Interest rates based on LTV (95%, 90%, 85%, 80%)
deposit_percentages = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40]
interest_rates = [0.0499, 0.0424, 0.0404, 0.0402, 0.0389, 0.0389, 0.0389, 0.038]  # Rates from market reference (compare the market best offer sorted by initial term cost)

compare_deposit_percentages(800000, deposit_percentages, interest_rates)


Property Price: £800,000
SDLT: £30,000 (First-time buyer relief)
 Deposit %   | Deposit Amount   | Loan Amount   | Rate   | Monthly (IO)   | Return on Extra Deposit   | SDLT + Deposit   | Funds/Cost Ratio
-------------+------------------+---------------+--------+----------------+---------------------------+------------------+--------------------
 5%          | £40,000          | £760,000      | 4.99%  | £3,160         | -                         | £70,000          | 8.75%
 10%         | £80,000          | £720,000      | 4.24%  | £2,544         | 18.49%                    | £110,000         | 13.75%
 15%         | £120,000         | £680,000      | 4.04%  | £2,289         | 7.64%                     | £150,000         | 18.75%
 20%         | £160,000         | £640,000      | 4.02%  | £2,144         | 4.36%                     | £190,000         | 23.75%
 25%         | £200,000         | £600,000      | 3.89%  | £1,945         | 5.97%                     | £230,000         | 28.75%
 3

## Example 3: Custom Analysis

Analyze different property prices and deposit percentages.

In [6]:
# Customize these values for your scenario
PROPERTY_PRICES = [600000, 700000, 800000, 900000]
DEPOSIT_PERCENTAGES = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40]

for price in PROPERTY_PRICES:
    compare_deposit_percentages(price, DEPOSIT_PERCENTAGES, interest_rates)

ValueError: interest_rates length (8) must match deposit_percentages length (5)

## Example 4: Find Maximum Affordable Property

Determine the maximum property price you can afford based on savings and loan constraints.

In [ ]:
# Find maximum affordable property
my_savings = 150000  # Available cash
deposit_percentage = 0.15  # 15% down payment
max_loan = 500000  # Maximum loan amount

max_price = find_max_property_price(my_savings, deposit_percentage, max_loan)
print(f"\nWith £{my_savings:,} savings, {deposit_percentage*100:.0f}% deposit, and max loan of £{max_loan:,}:")
print(f"Maximum affordable property price: £{max_price:,}" if max_price else "No affordable properties in range")